In [46]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
import sys, getopt
import csv

In [47]:
dataset=pd.read_csv("FootballStadiums.csv")
dataset.head
del dataset["Confederation"]
del dataset["City"]
del dataset["IOC"]
dataset['Stadium'] = dataset['Stadium'].str.replace(' ','_')
print(dataset)


                       Stadium                     HomeTeams  Capacity  \
0          Stadiumi_Besëlidhja                    Besëlidhja      7000   
1          Stadiumi_Flamurtari              Flamurtari Vlorë      8200   
2                Stadiumi_Laçi                       KF Laçi      5000   
3         Stadiumi_Niko_Dovana                         Teuta     12040   
4     Stadiumi_Selman_Stërmasi  KF Tirana, Dinamo, Partizani      9500   
...                        ...                           ...       ...   
2019           Liberty_Stadium                  Swansea City     21088   
2020           Parc_y_Scarlets        Scarlets, Llanelli RFC     14870   
2021               Park_Avenue              Aberystwyth Town      5000   
2022      Principality_Stadium                             -     74500   
2023         Racecourse_Ground                   Wrexham AFC     15850   

      Country  Population  
0     Albania     2876591  
1     Albania     2876591  
2     Albania     2876591  

In [48]:
stadium=[]
bio=[]
hometeams=[]
capacity=[]
country=[]
img=[]
bioConcat=[]
indice = 0

for i in dataset['Stadium']:
    
    bio = []
    
    res = requests.get("https://fr.wikipedia.org/wiki/"+str(i)+"")
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), "html.parser")
    all_ps = soup.find("div", class_="mw-parser-output")
    
    try:
        paragraphs = all_ps.find_all("p")
        urlimage = all_ps.find("a", class_="image")
        image = urlimage.find("img").attrs["src"]
       
    except AttributeError:
        indice+=1
        continue

    for paragraph in paragraphs: 
        if re.search("bandeau-titre", str(paragraph)) or re.search("Vous pouvez partager vos connaissances en l’améliorant",str(paragraph)) or re.search("navbar noprint bordered navigation-not-searchable", str(paragraph)) or re.search("Sur les autres projets Wikimedia :",str(paragraph)) or re.search("Aide:Insérer une référence",str(paragraph)) or re.search("bandeau-container homonymie plainlinks hatnote", str(paragraph)) or re.search("Pour les articles homonymes" ,str(paragraph)):
            continue
        
        else :
            bio.append(paragraph.text)

   
    if re.search(".png",str(image)):
        image = "Inconnue"

    stadium.append(i)
    bioConcat.append("".join(bio))
    img.append(image)
    capacity.append(dataset["Capacity"][indice])
    country.append(dataset["Country"][indice])
    indice+=1
    
    
    

df=pd.DataFrame({"Stadium":stadium,"Biographie":bioConcat,"url de l'image":img,"Capacity":capacity,"Country":country})

df

,Stadium,Biographie,url de l'image,Capacity,Country
0,Stade_Ahmed_Zabana,Le stade Ahmed-Zabana (en arabe : ملعب أحمد ز...,//upload.wikimedia.org/wikipedia/commons/thumb...,40000,Algeria
1,Stade_de_l'Unité_Maghrébine,Le Stade de l'Unité Maghrébine (en arabe : ملع...,Inconnue,22000,Algeria
2,Stade_Mohamed_Boumezrag,Le stade Mohamed-Boumezrag (en arabe : ملعب مح...,//upload.wikimedia.org/wikipedia/commons/thumb...,17000,Algeria
3,Stade_Mohamed-Benhaddad,Le stade Mohamed Benhaddad (en arabe : ملعب مح...,Inconnue,10000,Algeria
4,Stade_Omar_Hamadi,Le stade Omar-Hamadi (en arabe : ملعب عمر حماد...,Inconnue,12000,Algeria
...,...,...,...,...,...
797,Cardiff_City_Stadium,Le Cardiff City Stadium (Stadiwm Dinas Caerdyd...,Inconnue,32898,Wales
798,Parc_y_Scarlets,Le Parc y Scarlets (Scarlets Park en anglais) ...,Inconnue,14870,Wales
799,Park_Avenue,\nPark Avenue (autrefois Quatrième Avenue) est...,//upload.wikimedia.org/wikipedia/commons/thumb...,5000,Wales
800,Principality_Stadium,"Pour le parc de Chicago, voir Millennium Park....",Inconnue,74500,Wales


In [49]:
dataset["Capacity"][5]

7500

In [50]:
df.to_csv("wikiStade.csv",index=False)